In [1]:
import cv2
import numpy as np
from collections import deque
from ultralytics import YOLO
import torch

In [2]:
# ------------------------------
# YOLO setup
# ------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

yolo_model = YOLO("yolov8n.pt")   # nano = fast & sufficient
yolo_model.to(DEVICE)

# COCO class IDs
PERSON_ID = 0
BALL_ID = 32  # sports ball

Using device: cuda


In [3]:
from ultralytics import YOLO
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# Load YOLOv8 Nano (fast, sufficient for people detection)
yolo_model = YOLO("yolov8n.pt")
yolo_model.to(DEVICE)

print("YOLO loaded successfully")

Using device: cuda
YOLO loaded successfully


In [ ]:
# ------------------------------
# MANUAL COURT & NET GEOMETRY
# ------------------------------

# Court trapezoid (clockwise):
# top-left, top-right, bottom-right, bottom-left
COURT_POLY = np.array([
    [300, 200],  # Top-Left point
    [1000, 200], # Top-Right point
    [1200, 700], # Bottom-Right point
    [100, 700]   # Bottom-Left point
], dtype=np.int32)

# Net top edge (left pole → right pole)
'''NET_LINE = np.array([
    [344, 468],  # net left pole
    [1564, 469],  # net right pole
], dtype=np.int32)'''
# ------------------------------
# TEAM SEPARATION LINE
# ------------------------------
SPLIT_Y = 560  # <-- YOU SET THIS MANUALLY FROM THE IMAGE

In [ ]:
'''def point_side_of_line(p, a, b):
    """
    Returns:
        > 0 : one side of the net
        < 0 : other side of the net
    """
    return (b[0] - a[0]) * (p[1] - a[1]) - (b[1] - a[1]) * (p[0] - a[0])'''


In [11]:
def apply_court_mask(frame):
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [COURT_POLY], 255)
    return cv2.bitwise_and(frame, frame, mask=mask)


In [12]:
# ------------------------------
# Video paths
# ------------------------------
VIDEO_PATH = "/home/naman/Task-0/volleyball_match.mp4"
OUTPUT_PATH = "/home/naman/Task-0/volleyball_output.mp4"

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError("Cannot open video")

FPS = int(cap.get(cv2.CAP_PROP_FPS))
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(
    OUTPUT_PATH,
    cv2.VideoWriter_fourcc(*"mp4v"),
    FPS,
    (W, H)
)


In [ ]:
# ------------------------------
# 5. SAFETY CUT-OFFS
# ------------------------------
MAX_FRAMES = 2000      # hard cutoff to prevent kernel death
FRAME_SKIP = 1         # increase to 2 or 3 if GPU is weak

# ------------------------------
# 6. BALL TRAJECTORY
# ------------------------------
ball_trail = deque(maxlen=40)

# ------------------------------
# 7. MAIN LOOP (STABLE)
# ------------------------------
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count > MAX_FRAMES:
        print("Frame cutoff reached. Stopping safely.")
        break

    if frame_count % FRAME_SKIP != 0:
        continue

    # Court mask
    court_frame = apply_court_mask(frame)

    # YOLO inference
    results = yolo_model(
        court_frame,
        conf=0.30,
        device=DEVICE,
        verbose=False
    )

    near_count = 0
    far_count = 0
    ball_center = None

    for r in results:
        if r.boxes is None:
            continue

        for box in r.boxes:
            cls = int(box.cls[0])
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)

            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2

            # -------- PLAYER --------
           ''' if cls == PERSON_ID:
               side = point_side_of_line(
                    (cx, cy),
                    NET_LINE[0],
                    NET_LINE[1]
                )

                if side > 0:
                    near_count += 1
                    color = (0, 255, 0)   # near
                else:
                    far_count += 1
            
                    color = (255, 0, 0)   # far

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)'''

            # -------- BALL --------
            elif cls == BALL_ID:
                ball_center = (cx, cy)
                cv2.circle(frame, ball_center, 6, (0, 255, 255), -1)

    # -------- BALL TRAJECTORY --------
    if ball_center is not None:
        ball_trail.append(ball_center)

    for i in range(1, len(ball_trail)):
        cv2.line(frame, ball_trail[i-1], ball_trail[i], (0, 255, 255), 2)

    # -------- OVERLAYS --------
    cv2.polylines(frame, [COURT_POLY], True, (255, 255, 255), 2)
    cv2.line(frame, tuple(NET_LINE[0]), tuple(NET_LINE[1]), (0, 0, 255), 2)

    cv2.putText(frame, f"Near side players: {near_count}",
                (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
    cv2.putText(frame, f"Far side players: {far_count}",
                (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

    out.write(frame)

# ------------------------------
# 8. CLEAN EXIT
# ------------------------------
cap.release()
out.release()

print("Processing complete.")
print("Output saved to:", OUTPUT_PATH)

Frame cutoff reached. Stopping safely.
Processing complete.
Output saved to: /home/naman/Task-0/volleyball_output.mp4


In [2]:
# ============================================================
# FINAL STABLE YOLO-ONLY VOLLEYBALL ANALYSIS (NOTEBOOK SAFE)
# ============================================================

import cv2
import numpy as np
import torch
from ultralytics import YOLO
from collections import deque

# ------------------------------
# 1. DEVICE & MODEL
# ------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

yolo_model = YOLO("yolov8n.pt")
yolo_model.to(DEVICE)

# COCO class IDs
PERSON_ID = 0
BALL_ID = 32  # sports ball

# ------------------------------
# 2. MANUAL GEOMETRY (YOU EDIT)
# ------------------------------
# Court trapezoid (clockwise):
# top-left, top-right, bottom-right, bottom-left
COURT_POLY = np.array([
  #  [300, 200],  # Top-Left point
  #  [1000, 200], # Top-Right point
  #  [1200, 700], # Bottom-Right point
  #  [100, 700]   # Bottom-Left point
     [313,292],
     [996,289],
     [1198,708],
     [125,707]
], dtype=np.int32)

# Horizontal split line (top of net)
# Players ABOVE → far side
# Players BELOW → near side
SPLIT_Y = 453   # <-- PUT YOUR VALUE HERE

# ------------------------------
# 3. COURT MASK
# ------------------------------
def apply_court_mask(frame):
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [COURT_POLY], 255)
    return cv2.bitwise_and(frame, frame, mask=mask)

# ------------------------------
# 4. BALL FILTER (IMPORTANT)
# ------------------------------
def is_valid_ball(bbox, frame_area):
    x1, y1, x2, y2 = bbox
    area = (x2 - x1) * (y2 - y1)

    if area < 50:                     # too small → noise
        return False
    if area > 0.01 * frame_area:      # too large → not ball
        return False

    return True

# ------------------------------
# 5. VIDEO I/O (HEADLESS)
# ------------------------------
VIDEO_PATH = "/home/naman/Cryptonite-RTP-NamanGoel/Task-0/volleyball_match.mp4"
OUTPUT_PATH = "/home/naman/Cryptonite-RTP-NamanGoel/Task-0/volleyball_match_output_YOLO_only.mp4"

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError("Cannot open video")

FPS = int(cap.get(cv2.CAP_PROP_FPS))
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
FRAME_AREA = W * H

out = cv2.VideoWriter(
    OUTPUT_PATH,
    cv2.VideoWriter_fourcc(*"mp4v"),
    FPS,
    (W, H)
)

# ------------------------------
# 6. SAFETY CUT-OFFS
# ------------------------------
MAX_FRAMES = 2000     # prevents kernel death
FRAME_SKIP = 1        # increase to 2 if GPU is weak

# ------------------------------
# 7. BALL TRAJECTORY
# ------------------------------
ball_trail = deque(maxlen=40)

# ------------------------------
# 8. MAIN LOOP (STABLE)
# ------------------------------
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count > MAX_FRAMES:
        print("Frame cutoff reached. Stopping safely.")
        break

    if frame_count % FRAME_SKIP != 0:
        continue

    # Apply court mask
    court_frame = apply_court_mask(frame)

    # YOLO inference (LOW confidence for ball recall)
    results = yolo_model(
        court_frame,
        conf=0.15,
        device=DEVICE,
        verbose=False
    )

    near_count = 0
    far_count = 0

    best_ball = None
    best_ball_area = 0

    for r in results:
        if r.boxes is None:
            continue

        for box in r.boxes:
            cls = int(box.cls[0])
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)

            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2

            # -------- PLAYER --------
            if cls == PERSON_ID:
                if cy > SPLIT_Y:
                    near_count += 1
                    color = (0, 255, 0)   # near side
                else:
                    far_count += 1
                    color = (255, 0, 0)   # far side

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            # -------- BALL --------
            elif cls == BALL_ID:
                if not is_valid_ball((x1, y1, x2, y2), FRAME_AREA):
                    continue

                area = (x2 - x1) * (y2 - y1)
                if area > best_ball_area:
                    best_ball_area = area
                    best_ball = (cx, cy)

    # -------- BALL TRAJECTORY --------
    if best_ball is not None:
        ball_trail.append(best_ball)
        cv2.circle(frame, best_ball, 6, (0, 255, 255), -1)

    for i in range(1, len(ball_trail)):
        cv2.line(frame, ball_trail[i-1], ball_trail[i], (0, 255, 255), 2)

    # -------- OVERLAYS --------
    cv2.polylines(frame, [COURT_POLY], True, (255, 255, 255), 2)
    cv2.line(frame, (0, SPLIT_Y), (W, SPLIT_Y), (0, 0, 255), 2)

    cv2.putText(frame, f"Near side players: {near_count}",
                (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
    cv2.putText(frame, f"Far side players: {far_count}",
                (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

    out.write(frame)

# ------------------------------
# 9. CLEAN EXIT
# ------------------------------
cap.release()
out.release()

print("Processing complete.")
print("Output saved to:", OUTPUT_PATH)


Using device: cuda
Frame cutoff reached. Stopping safely.
Processing complete.
Output saved to: /home/naman/Cryptonite-RTP-NamanGoel/Task-0/volleyball_match_output_YOLO_only.mp4
